# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [1]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr


%matplotlib notebook

In [2]:
stl_file = 'roboy_models/TestCube/stls/sphere224.stl'

In [3]:
#!/usr/bin/env python  
import roslib
#roslib.load_manifest('rqt_gui_cpp_node_5974')
import rospy
import math
import tf
#import geometry_msgs
#import turtlesim.srv

rospy.init_node('lighthouse_tf_listener')

listener1 = tf.TransformListener()
listener2 = tf.TransformListener()

rate = rospy.Rate(10.0)
while not rospy.is_shutdown():
    try:
        (trans1,rot1) = listener1.lookupTransform('/world', '/lighthouse1', rospy.Time(0))
        (trans2,rot2) = listener2.lookupTransform('/world', '/lighthouse2', rospy.Time(0))
    except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
        continue

    print(trans1);print(rot1)
    print(trans2);print(rot2)
    print("")
    rate.sleep()
    
    break

[-2.0, 0.0, 2.0]
[0.0, 0.0, -0.7071067811865475, 0.7071067811865476]
[2.0, 0.0, 2.0]
[0.0, 0.0, 0.7071067811865475, 0.7071067811865476]



In [4]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=-np.pi / 2)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[0,0,1], angle=np.pi / 2)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

([-2.0, 0, 2.0], Quaternion(0.70710678118654757, -0.0, -0.0, -0.70710678118654746))
([2, 0.0, 2.0], Quaternion(0.70710678118654757, 0.0, 0.0, 0.70710678118654746))


In [5]:
from roboy_communication_middleware.msg import LighthousePoseCorrection

pub = rospy.Publisher('roboy/middleware/DarkRoom/LighthousePoseCorrection', LighthousePoseCorrection, queue_size=1)

#Define Position LIGHTHOUSE 1
msg = LighthousePoseCorrection()


In [6]:
msg.type = 1 #1 for absolut position, 0 for relative position
msg.id = 0

msg.tf.translation.x = translationLH1[0]
msg.tf.translation.y = translationLH1[1]
msg.tf.translation.z = translationLH1[2]

msg.tf.rotation.x = quat1[1]
msg.tf.rotation.y = quat1[2]
msg.tf.rotation.z = quat1[3]
msg.tf.rotation.w = quat1[0]

pub.publish(msg)

In [7]:
#Define Position LIGHTHOUSE 2
msg.id = 1
msg.tf.translation.x = translationLH2[0]
msg.tf.translation.y = translationLH2[1]
msg.tf.translation.z = translationLH2[2]

msg.tf.rotation.x = quat2[1]
msg.tf.rotation.y = quat2[2]
msg.tf.rotation.z = quat2[3]
msg.tf.rotation.w = quat2[0]

pub.publish(msg)

In [8]:
from visualization_msgs.msg import Marker, MarkerArray
from data.trafomatrix import get_random_trafomatrix

scale = 0.01
duration = 1
pos = [0,0,0]
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)


#Visualization mesh
topic = 'visualization_marker'
publisher = rospy.Publisher(topic, Marker, queue_size=1)

rate2 = rospy.Rate(1)


In [9]:
while not rospy.is_shutdown():
    
    for i in range(3):
        #Transform mesh
        #T = get_random_trafomatrix()
        #orientationMesh = Quaternion(matrix=T)

        mesh = Marker()
        mesh.header.frame_id = 'world';
        mesh.ns = 'ns';
        mesh.type = Marker.MESH_RESOURCE
        mesh.color.r = 1.0
        mesh.color.g = 1.0
        mesh.color.b = 1.0
        mesh.color.a = 0.9
        mesh.scale.x = scale
        mesh.scale.y = scale
        mesh.scale.z = scale
        mesh.lifetime = rospy.Duration(-1);
        mesh.header.stamp = rospy.Time.now()
        mesh.action = Marker.ADD
        mesh.id = 10000000;
        mesh.pose.position.x = pos[0];
        mesh.pose.position.y = pos[1];
        mesh.pose.position.z = pos[2];
        mesh.pose.orientation.x = orientationMesh[1];
        mesh.pose.orientation.y = orientationMesh[2];
        mesh.pose.orientation.z = orientationMesh[3];
        mesh.pose.orientation.w = orientationMesh[0];
        mesh.mesh_resource = ("package://" + stl_file)
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(mesh);
        rate2.sleep()
        #MeshMarker.type = visualization_msgs::Marker::MESH_RESOURCE;
        #MeshMarker.mesh_resource = "package://pr2_description/meshes/base_v0/base.dae";
    break
    #print(mesh)

In [10]:
rospy.init_node('lighthouse_tf_listener')

listener1 = tf.TransformListener()
listener2 = tf.TransformListener()

rate = rospy.Rate(10.0)
while not rospy.is_shutdown():
    try:
        (trans1,rot1) = listener1.lookupTransform('/world', '/lighthouse1', rospy.Time(0))
        (trans2,rot2) = listener2.lookupTransform('/world', '/lighthouse2', rospy.Time(0))
    except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
        continue
    print('Lighthouse 1:')
    print('Translation: %s' %trans1)
    print('Rotation: %s' %rot1)
    print('')
    print('Lighthouse 2:')
    print('Translation: %s' %trans2)
    print('Rotation: %s' %rot2)
    
    rate.sleep()
    
    break

Lighthouse 1:
Translation: [-2.0, 0.0, 2.0]
Rotation: [0.0, 0.0, -0.7071067811865475, 0.7071067811865476]

Lighthouse 2:
Translation: [2.0, 0.0, 2.0]
Rotation: [0.0, 0.0, 0.7071067811865475, 0.7071067811865476]


# Preprocess data

In [11]:
#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../src/'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles 
triangles = scale * np.matrix(mesh1.points)


vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#normals = np.zeros_like((len(triangles),3))
normalsNotNorm = mesh1.normals

normals = []
for normal in normalsNotNorm:
    normals.append(1/np.linalg.norm(normal,2)*normal)
    
normals = np.matrix(normals)
normals = scale * normals

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
#print(triangles)
print('')
print('%d vertices' %len(vertices))
#print(vertices)
print('')
print('%d normals normiert' %len(normals))
#print(normals)
print('')
print('%d sensors in centers of triangles' %len(sensors))
#print(sensors)
#print(vertices); print(normals); print(sensors)

224 triangles

672 vertices

224 normals normiert

224 sensors in centers of triangles


In [12]:
#Prepare vertices, normals and sensor locations for Transformation

verticesRot = np.insert(vertices, 3,values=1,axis=1)
print(verticesRot)
print('')

normalsRot = np.insert(normals, 3, values=0, axis=1)
print(normalsRot)
print('')

sensorsRot = np.insert(vertices, 3,values=1,axis=1)
print(sensorsRot)
print('')

[[ -5.72377014e-02   2.42112732e-02  -1.16002989e-01   1.00000000e+00]
 [ -2.30117798e-02  -7.60902405e-03  -1.34323759e-01   1.00000000e+00]
 [ -7.21518707e-02  -1.52587891e-07  -1.07804518e-01   1.00000000e+00]
 ..., 
 [ -3.49208832e-03  -9.99381924e-02  -2.70083880e-02   1.00000000e+00]
 [ -6.60676956e-04  -9.94094467e-02  -4.85675335e-02   1.00000000e+00]
 [  2.06223106e-02  -9.85083485e-02  -4.05343151e-02   1.00000000e+00]]

[[ -4.75511312e-03  -4.96887323e-05  -8.79695654e-03   0.00000000e+00]
 [ -9.87434804e-03   3.65564600e-04   1.53740928e-03   0.00000000e+00]
 [  3.12507264e-04   7.22206086e-04   9.96899009e-03   0.00000000e+00]
 [  9.87468779e-03   3.26725803e-04  -1.54395431e-03   0.00000000e+00]
 [  4.76005256e-03   1.40244782e-03   8.68187964e-03   0.00000000e+00]
 [  5.89473426e-03   7.55205974e-04   8.04249942e-03   0.00000000e+00]
 [  8.21867645e-03   1.42911509e-03  -5.51461637e-03   0.00000000e+00]
 [  2.63164967e-03   1.03196383e-03  -9.59215641e-03   0.00000000e+0

In [13]:
#Test transform vertices
T = get_random_trafomatrix()
newvertices = np.dot(T,np.transpose(verticesRot)).T


#print(newvertices)

In [14]:
#Sensor visualization in RVIZ


for i,sensor in enumerate(sensors):

    sensor = np.squeeze(np.array(sensor))
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = sensor[0];
        markers.pose.position.y = sensor[1];
        markers.pose.position.z = sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 0.0
        markers.color.g = 0.0
        markers.color.b = 1.0
        markers.color.a = 1.0

        markers.scale.x = 0.01
        markers.scale.y = 0.01
        markers.scale.z = 0.01
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i

        #mesh.mesh_resource = "package://common_utilities/stls/testcube_40mm.stl"
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(markers);
        rate3.sleep()
        break


# GA

In [15]:
from deap import algorithms, base, creator, tools

In [16]:
#sensors to dict
sensorList = []

for sensor in sensors:
    sensor = np.squeeze(np.array(sensor)).tolist()
    sensorList.append(sensor)

sensor_dict =  list(zip(range(len(sensors)), sensorList))

In [17]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,1.0,1.0)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [18]:
toolbox = base.Toolbox()

In [19]:
# Find random sensors
def randomSensor(v):
    i = random.randint(0,len(v)-1)
    #print(i)
    rand = v[i]
    
    return rand[0]

In [20]:
r = np.squeeze(np.array(randomSensor(sensors)))

print(r)

[ 0.09394256  0.0210256  -0.0575987 ]


In [21]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 8)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [22]:
toolbox.attr_bool()

59

In [23]:
toolbox.individual()

[175, 220, 34, 183, 91, 81, 172, 156]

In [24]:
toolbox.population(n=30)

[[79, 134, 104, 65, 77, 51, 133, 121],
 [42, 116, 13, 101, 111, 11, 160, 166],
 [13, 202, 175, 26, 1, 219, 87, 73],
 [190, 193, 90, 218, 191, 22, 11, 162],
 [115, 105, 173, 181, 64, 164, 35, 153],
 [221, 79, 73, 216, 7, 115, 121, 29],
 [162, 78, 86, 216, 19, 151, 211, 106],
 [32, 28, 223, 0, 17, 176, 113, 138],
 [183, 128, 101, 31, 135, 185, 115, 90],
 [155, 52, 126, 182, 150, 59, 133, 43],
 [106, 165, 123, 206, 125, 172, 206, 145],
 [158, 40, 45, 138, 64, 115, 54, 176],
 [48, 209, 91, 98, 143, 5, 85, 12],
 [113, 33, 143, 160, 105, 21, 191, 61],
 [207, 2, 71, 191, 90, 76, 70, 45],
 [199, 191, 165, 203, 138, 151, 27, 186],
 [84, 183, 11, 35, 27, 113, 95, 181],
 [181, 135, 168, 204, 168, 108, 197, 88],
 [183, 40, 71, 149, 215, 188, 197, 107],
 [109, 211, 221, 6, 53, 164, 50, 107],
 [160, 116, 12, 36, 67, 102, 159, 159],
 [63, 196, 101, 62, 110, 192, 12, 59],
 [92, 195, 22, 176, 205, 64, 0, 13],
 [121, 161, 133, 170, 108, 26, 153, 153],
 [145, 107, 214, 209, 187, 62, 127, 169],
 [187, 126

# Evaluation (Fitness) Function

In [25]:
from data.RayIntersectsTriangle import rayIntersectsTriangle,rayIntersectsTriangleVisualization

def FitnessFunction(sensors):
    
    #print(sensors)
    #print(sensor_dict)
    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0
    visibleLH2 = 0

    for i, nmbr_sensor in enumerate(sensors):
        #print(nmbr_sensor)
        sensor = sensor_dict[nmbr_sensor][1]
        #print(sensor)
        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[i])), 'lighthouse1')
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[i])), 'lighthouse2')
        #print(interDistLH1)
        #print(interDistLH2)

        for j in range(len(triangles)):
            if(i != j):
                #print(j)
                newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse1')
                newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse2')
                if(newInterDistLH1 > interDistLH1 and newInterDistLH1 != False):
                    interDistLH1 = newInterDistLH1
                    visibleLH1 += 1

                if(newInterDistLH2 > interDistLH2 and newInterDistLH2 != False):
                    interDistLH2 = newInterDistLH2
                    visibleLH2 += 1
    
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                dist += np.linalg.norm(ind-indivi)
    
    #print(visibleLH1);print(visibleLH2);print('')
    
    return dist, visibleLH1, visibleLH2

In [26]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensorList)-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [27]:
# Creating population

population = toolbox.population(n=10)

In [28]:
hof = tools.HallOfFame(1)

In [29]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [30]:
population, log = algorithms.eaSimple(population, 
                                      toolbox, 
                                      cxpb=0.5, 
                                      mutpb=0.2, 
                                      ngen=1000, 
                                      stats=stats, 
                                      halloffame=hof, 
                                      verbose=True)

gen	nevals	avg    	std    	min   	max
0  	10    	9.61566	4.22613	3.4617	14 
1  	10    	9.4676 	4.17114	3.26643	15 
2  	3     	9.52075	4.11911	3.5382 	14 
3  	5     	9.53562	4.10646	3.65917	15 
4  	6     	9.74957	4.20048	3.82344	14 
5  	6     	9.78778	4.19154	3.8781 	14 
6  	8     	9.75397	4.17845	3.85122	14 
7  	7     	9.65355	4.07082	3.86917	14 
8  	4     	9.61646	4.04439	3.83287	14 
9  	9     	9.54115	4.05796	3.72486	14 
10 	7     	9.81488	4.18202	3.81649	14 
11 	2     	9.66039	4.03651	3.98117	13 
12 	6     	9.51613	3.97176	3.76772	13 
13 	7     	9.66039	4.03651	3.98117	13 
14 	6     	9.7545 	4.13769	3.80441	15 
15 	8     	9.68607	4.10334	3.75163	15 
16 	8     	9.68891	4.07479	3.8369 	14 
17 	8     	9.72018	4.10057	3.77476	14 
18 	2     	9.66253	4.10697	3.03104	13 
19 	6     	9.65727	4.04099	3.83056	13 
20 	10    	9.68604	4.07065	3.86385	13 
21 	5     	9.54772	4.02396	3.75784	14 
22 	7     	9.69929	4.06015	3.99543	14 
23 	7     	9.79639	4.17958	3.92316	15 
24 	2     	9.78419	4.19775	

210	5     	9.47218	4.17017	3.88982	15 
211	5     	9.38939	4.08935	4.01843	14 
212	5     	9.44936	4.12383	3.93409	14 
213	8     	9.30202	4.0737 	3.66604	14 
214	4     	9.38833	4.12321	3.98667	15 
215	2     	9.3579 	4.07776	4.07369	14 
216	6     	9.40708	4.15153	3.67839	15 
217	2     	9.3579 	4.07776	4.07369	14 
218	4     	9.30038	4.08436	3.52525	14 
219	6     	9.37593	4.09137	3.61474	14 
220	6     	9.17709	4.02121	3.64949	14 
221	7     	9.28447	4.0178 	3.92085	14 
222	7     	9.38855	4.12292	3.99326	15 
223	8     	9.40319	4.23564	3.87142	15 
224	4     	9.44593	4.16046	3.98434	15 
225	6     	9.41701	4.10577	3.94494	14 
226	10    	9.44474	4.18439	3.33804	15 
227	8     	9.43526	4.19069	3.77104	16 
228	6     	9.35819	4.07737	4.07369	14 
229	1     	9.35467	4.08199	3.95917	14 
230	3     	9.30887	4.04787	3.84675	14 
231	9     	9.36802	4.11774	3.72918	15 
232	8     	9.3205 	4.04897	3.95369	14 
233	8     	9.47483	4.16684	3.83133	15 
234	9     	9.48182	4.16553	3.92003	15 
235	5     	9.37767	4.1131

421	8     	9.98278	4.30212	3.93102	15 
422	4     	9.98861	4.27854	3.88121	14 
423	8     	10.0871	4.36072	3.91356	16 
424	8     	9.98566	4.26718	3.79266	14 
425	10    	10.0397	4.34762	3.41509	15 
426	8     	9.90592	4.22962	3.85807	14 
427	2     	10.047 	4.34396	3.84536	15 
428	3     	9.99539	4.25336	4.08458	14 
429	8     	10.0288	4.28053	4.08636	14 
430	6     	10.083 	4.34362	3.78289	14 
431	5     	9.94913	4.25186	3.76609	14 
432	7     	9.98937	4.27747	3.90376	14 
433	4     	9.99199	4.27376	3.9824 	14 
434	6     	10.0585	4.30462	3.9788 	14 
435	4     	10.02  	4.29303	3.82145	14 
436	5     	10.02  	4.28509	3.94614	14 
437	1     	10.0588	4.31965	3.98792	15 
438	5     	10.0288	4.28053	4.08636	14 
439	6     	10.1148	4.37182	3.7933 	15 
440	7     	9.98768	4.25634	3.94273	14 
441	8     	10.0162	4.29039	3.92521	14 
442	6     	10.0193	4.2862 	3.80203	14 
443	5     	10.1084	4.37374	3.55114	14 
444	5     	10.0263	4.29179	4.00979	14 
445	6     	9.91813	4.24394	3.94012	14 
446	7     	10.0392	4.3626

632	5     	9.72872	4.01882	4.03856	14 
633	8     	9.74675	4.06376	3.66662	14 
634	0     	9.69715	3.98479	4.09146	13 
635	10    	9.7225 	4.03593	3.9322 	14 
636	7     	9.68908	4.00465	3.87669	14 
637	6     	9.71969	4.03167	3.90498	14 
638	4     	9.69029	3.9946 	3.8856 	13 
639	6     	9.6942 	3.99731	4.04263	14 
640	4     	9.75867	4.04644	3.93694	14 
641	5     	9.58528	3.9464 	3.90881	13 
642	6     	9.62635	3.95139	3.96719	13 
643	6     	9.68667	3.99989	3.78107	13 
644	7     	9.65135	3.98728	3.90054	13 
645	5     	9.47399	3.90359	3.89113	13 
646	6     	9.6515 	3.99538	3.89113	14 
647	5     	9.68973	4.03682	3.97476	14 
648	7     	9.67915	4.04345	3.87512	14 
649	3     	9.68845	4.0138 	3.93274	14 
650	5     	9.65867	3.96847	3.93685	13 
651	5     	9.65404	4.03327	3.9453 	14 
652	6     	9.65441	4.0246 	3.85287	14 
653	8     	9.61103	3.99017	3.75528	13 
654	7     	9.75142	4.10577	3.88969	15 
655	8     	9.61822	3.96291	3.90735	13 
656	6     	9.69197	3.99218	3.9359 	13 
657	6     	9.69428	3.9888

843	2     	9.75724	4.04843	3.99218	14 
844	8     	9.57627	3.95927	3.77681	13 
845	6     	9.62046	3.97649	3.95684	14 
846	9     	9.56055	3.92633	3.99339	13 
847	8     	9.65574	3.98936	3.92602	14 
848	8     	9.7492 	4.06036	3.65289	14 
849	5     	9.69579	3.98671	4.05051	13 
850	5     	9.6517 	3.97842	3.86304	13 
851	9     	9.61161	3.97267	3.68686	13 
852	7     	9.7066 	4.05881	3.74647	14 
853	7     	9.5861 	3.95389	3.75979	13 
854	3     	9.69601	3.98641	4.05705	13 
855	8     	9.77713	4.09022	3.78937	14 
856	4     	9.65525	3.98182	3.83441	13 
857	5     	9.55761	4.00237	3.76569	14 
858	7     	9.62574	3.95221	4.00019	13 
859	8     	9.74632	4.06423	3.72629	14 
860	4     	9.7466 	4.07183	3.80604	14 
861	1     	9.69682	3.98526	4.08142	13 
862	8     	9.6542 	4.00839	3.80268	14 
863	7     	9.72614	4.01421	3.96107	13 
864	6     	9.54123	3.97911	3.71393	13 
865	6     	9.72577	4.03128	3.99104	14 
866	5     	9.72911	4.00997	4.05022	13 
867	5     	9.65755	3.97009	3.90331	13 
868	5     	9.75729	4.0401

In [31]:
best_sensors = tools.selBest(population, k=1)[0]
print(best_sensors)

[166, 173, 46, 117, 29, 20, 107, 201]


In [32]:
#Sensor visualization in RVIZ

for i, best in enumerate(best_sensors):

    show_sensor = sensor_dict[best][1]
    
    
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = show_sensor[0];
        markers.pose.position.y = show_sensor[1];
        markers.pose.position.z = show_sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 1.0
        markers.color.g = 0.0
        markers.color.b = 0.0
        markers.color.a = 1.0

        markers.scale.x = 0.015
        markers.scale.y = 0.015
        markers.scale.z = 0.015
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i

        #mesh.mesh_resource = "package://common_utilities/stls/testcube_40mm.stl"
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(markers);
        rate3.sleep()
        break
